### Installs and imports

In [26]:
!pip3 install tweepy flatdict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for flatdict: filename=flatdict-4.0.1-py3-none-any.whl size=6946 sha256=07ff5e2c60f2c99b75863648486863f64d92322d9bb92d7db2bb78dbcfe8f1e9
  Stored in directory: /root/.cache/pip/wheels/1a/f4/4e/e18d5704aab95b3da94c5d8251004b31068e69a144eb42403a
Successfully built flatdict


In [27]:
import csv
import tweepy
import ssl
import time
import pandas as pd
import numpy as np
import json
import flatdict

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context

### Declare authentication keys

In [4]:
# Twitter authentication keys
consumer_key = " "
consumer_secret = " "
access_token = " "
access_token_secret = " "

In [5]:
# Authentication with Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

### Scrape tweets from a specified user

For more parameters on `user_timeline` endpoint, see [HERE](https://www.geeksforgeeks.org/python-api-user_timeline-in-tweepy/).

In [12]:
my_tweets = api.user_timeline(screen_name='theJagmeetSingh', count=25)
for tweet in my_tweets:
  print (tweet.text)

Happy 87th Birthday to my friend and former NDP leader Ed Broadbent!

Thank you for being a progressive legend and… https://t.co/I234acX2Ib
Canadians deserve answers.

They need to know their votes count.

And that elections are safe from foreign interfer… https://t.co/2mLCD2ZrPS
À l’occasion de la Journée internationale pour l’élimination de la discrimination raciale, réaffirmons notre engage… https://t.co/7HVkP9VXJL
Everyday, people face harassment and are denied services based on how they look or where they’re from.

On the Inte… https://t.co/wvgkErWKRS
I would like to extend my best wishes to everyone celebrating Nowruz - the New Year in the Persian calendar.

And a… https://t.co/nvi6ccxF4a
RT @alexboulerice: Pendant que les libéraux se traînent les pieds, il y a des travailleurs et travailleuses qui souffrent. ⁦@PortQuebec⁩ ⁦@…
Bonjour tout le monde,

Si vous lisez ceci, je sais que les temps sont particulièrement difficiles pour beaucoup d'… https://t.co/OVsZqha7bz
Good morning,


### Scrape tweets from multiple handles

In [13]:
names = ['theJagmeetSingh']  # placeholder screen name for the handle you want to retrieve tweets from
# YOU CAN QUERY MULTIPLE HANDLES, BUT REMEMBER TO HANDLE TWITTER'S TIMEOUT/LIMIT CONSTRAINTS

In [14]:
""" Retrieves the most recent 200-ish tweets from a  list of given screen names
    Returns a data frame of the tweets with [id, screen_name, text, timestamp]
"""
def get_new_tweets(names, count):
  print("Retrieving tweets...")
  corpus = []                                                                                        
  for name in names:
    tweets = api.user_timeline(screen_name=name,  count=count)          
    time.sleep(4)
    corpus.extend(tweets)                                                                          
  
  data = [[tweet.id_str, tweet.user.screen_name, tweet.text, tweet.created_at] for tweet in corpus]

  tweets_df = pd.DataFrame(data, columns=['tweet_id', 'screen_name', 'text', 'timestamp'])      
            
  return tweets_df

In [15]:
tweets = get_new_tweets (names, count=30)
print (tweets)

Retrieving tweets...
               tweet_id      screen_name  \
0   1638295847971733504  theJagmeetSingh   
1   1638251977745530880  theJagmeetSingh   
2   1638241564643192858  theJagmeetSingh   
3   1638239643895230480  theJagmeetSingh   
4   1637915460422467584  theJagmeetSingh   
5   1637842292634718209  theJagmeetSingh   
6   1637841427496329218  theJagmeetSingh   
7   1637837433172533251  theJagmeetSingh   
8   1637174790594215944  theJagmeetSingh   
9   1637174786030747649  theJagmeetSingh   
10  1637174784155959296  theJagmeetSingh   
11  1637159432856453124  theJagmeetSingh   
12  1637106585565556736  theJagmeetSingh   
13  1637076775837507585  theJagmeetSingh   
14  1636827245057589259  theJagmeetSingh   
15  1636823038338682880  theJagmeetSingh   
16  1636816312919003136  theJagmeetSingh   
17  1636782040510087168  theJagmeetSingh   
18  1636776913938137090  theJagmeetSingh   
19  1636775224300503040  theJagmeetSingh   
20  1636760631247351809  theJagmeetSingh   
21  1636467

In [16]:
tweets.head()

,tweet_id,screen_name,text,timestamp
0,1638295847971733504,theJagmeetSingh,Happy 87th Birthday to my friend and former ND...,2023-03-21 21:45:42+00:00
1,1638251977745530880,theJagmeetSingh,Canadians deserve answers.\n\nThey need to kno...,2023-03-21 18:51:23+00:00
2,1638241564643192858,theJagmeetSingh,À l’occasion de la Journée internationale pour...,2023-03-21 18:10:00+00:00
3,1638239643895230480,theJagmeetSingh,"Everyday, people face harassment and are denie...",2023-03-21 18:02:22+00:00
4,1637915460422467584,theJagmeetSingh,I would like to extend my best wishes to every...,2023-03-20 20:34:11+00:00


### Scrape replies to a status

For definition of various Tweet objects see [here](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet).

In [6]:
def get_replies (user_name, tweet_id, num):
  
  replies = tweepy.Cursor (api.search_tweets, 
                          q='to:{}'.format(user_name),
                          since_id=tweet_id, 
                          tweet_mode='extended').items(num)
  while True:
    try:
      reply = replies.next()
      if not hasattr(reply, 'in_reply_to_status_id_str'):
        continue
      if reply.in_reply_to_status_id==tweet_id:
        logging.info("reply of tweet:{}".format(reply.full_text))

    #except tweepy.RateLimitError as e:
    #  logging.error("Twitter api rate limit reached".format(e))
    #  time.sleep(60)
    #  continue

    #except tweepy.TweepError as e:
    #  logging.error("Tweepy error occured:{}".format(e))
    #  break

    except StopIteration:
      break

    except Exception as e:
      logger.error("Failed while fetching replies {}".format(e))
      break
  
  return replies

**Alternative (updated) approach**: faster and neater

In [7]:
def extract_replies (name, tweet_id, num):

  replies=[]
  for tweet in tweepy.Cursor(api.search_tweets,q='to:'+name, since_id=tweet_id, tweet_mode='extended').items(num):
      if hasattr(tweet, 'in_reply_to_status_id_str'):
          if (tweet.in_reply_to_status_id_str==tweet_id):
              replies.append(tweet)

  with open('replies_clean.csv', 'w') as f:
      csv_writer = csv.DictWriter(f, fieldnames=('tweet_id', 'timestamp', 'user', 'text'))
      csv_writer.writeheader()
      for tweet in replies:
          row = {'tweet_id': tweet.id_str, 'timestamp':tweet.created_at, 'user': tweet.user.screen_name, 'text': tweet.full_text}
          csv_writer.writerow(row)

Extract replies corresponding to Jagmeet Singh's Twitter post

In [10]:
username = "theJagmeetSingh"
tweet_id = "1637174784155959296"

extract_replies (name=username, tweet_id=tweet_id, num=1000)

In [20]:
replies = pd.read_csv("replies_clean.csv") 
replies.head(20)

,tweet_id,timestamp,user,text
0,1638525221098364933,2023-03-22 12:57:09+00:00,silvergoodenuf,@theJagmeetSingh You supported this on Canadia...
1,1638522745011986432,2023-03-22 12:47:19+00:00,MichaelAddams,@theJagmeetSingh Jagmeet Singh you are parodyi...
2,1638522341524160515,2023-03-22 12:45:43+00:00,BaleNorm,@theJagmeetSingh Religion shows its worth once...
3,1638518193605029888,2023-03-22 12:29:14+00:00,Julzzz41,@theJagmeetSingh You only understand how civil...
4,1638517775563128832,2023-03-22 12:27:34+00:00,paneerholic,@theJagmeetSingh I call on Government of Canad...
5,1638515854798094336,2023-03-22 12:19:56+00:00,ManavKh00375779,@theJagmeetSingh When u going back to punjab p...
6,1638510576757530624,2023-03-22 11:58:58+00:00,Vancsun,@theJagmeetSingh Feels like you are their real...
7,1638502335407169536,2023-03-22 11:26:13+00:00,Oldsalt52,@theJagmeetSingh Go back sellout band confront...
8,1638436629563015169,2023-03-22 07:05:07+00:00,patshahidawa,@theJagmeetSingh #freeamritpal #freeamritpalsi...
9,1638407711451668481,2023-03-22 05:10:13+00:00,canapositivebee,"@theJagmeetSingh If u r so so concerned, go ba..."


### Extract account information of a user

In [36]:
user_info = api.get_user(screen_name='theJagmeetSingh')
user_info_json = json.loads(json.dumps (user_info._json, indent=2))
pd.DataFrame ([user_info_json]).columns

Index(['id', 'id_str', 'name', 'screen_name', 'location', 'profile_location',
       'description', 'url', 'entities', 'protected', 'followers_count',
       'friends_count', 'listed_count', 'created_at', 'favourites_count',
       'utc_offset', 'time_zone', 'geo_enabled', 'verified', 'statuses_count',
       'lang', 'status', 'contributors_enabled', 'is_translator',
       'is_translation_enabled', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_image_url',
       'profile_image_url_https', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image',
       'has_extended_profile', 'default_profile', 'default_profile_image',
       'following', 'follow_request_sent', 'notifications', 'translator_type',
       'withheld_scope', 'withheld_in_countries'],
      dtype='object')

In [38]:
user_info_json

{'id': 253340075,
 'id_str': '253340075',
 'name': '',
 'screen_name': 'theJagmeetSingh',
 'location': '',
 'profile_location': None,
 'description': "@theJagmeetSingh's account has been withheld in India in response to a legal demand. Learn more.",
 'url': None,
 'entities': {'description': {'urls': [{'url': 'https://support.twitter.com/articles/20169222',
     'expanded_url': 'https://support.twitter.com/articles/20169222',
     'display_url': 'Learn more',
     'indices': [85, 95]}]}},
 'protected': False,
 'followers_count': 597437,
 'friends_count': 1465,
 'listed_count': 1771,
 'created_at': 'Thu Feb 17 01:38:20 +0000 2011',
 'favourites_count': 2839,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'verified': True,
 'statuses_count': 16578,
 'lang': None,
 'status': {'created_at': 'Tue Mar 21 21:45:42 +0000 2023',
  'id': 1638295847971733504,
  'id_str': '1638295847971733504',
  'text': "@theJagmeetSingh's account has been withheld in India in response to a legal

In [39]:
pd.DataFrame ([user_info_json])

,id,id_str,name,screen_name,location,profile_location,description,url,entities,protected,...,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type,withheld_scope,withheld_in_countries
0,253340075,253340075,,theJagmeetSingh,,None,@theJagmeetSingh's account has been withheld i...,None,{'description': {'urls': [{'url': 'https://sup...,False,...,True,True,False,False,False,False,False,none,user,[IN]
